# Analiza podatkov s knjižnico Pandas

Spodaj je pregled najosnovnejših metod, ki jih ponuja knjižnica Pandas. Vsaka od naštetih metod ponuja še cel kup dodatnih možnosti, ki so natančno opisane v [uradni dokumentaciji](http://pandas.pydata.org/pandas-docs/stable/). Z branjem dokumentacije se vam seveda najbolj splača začeti pri [uvodih](http://pandas.pydata.org/pandas-docs/stable/tutorials.html).

### Predpriprava

In [ ]:
# naložimo paket
import pandas as pd

# naložimo razpredelnico, s katero bomo delali
filmi = pd.read_csv('obdelani-podatki/filmi.csv', index_col='id')

# ker bomo delali z velikimi razpredelnicami, povemo, da naj se vedno izpiše le 20 vrstic
pd.options.display.max_rows = 20

## Delo z razpredelnicami

### Osnovni izbori elementov razpredelnic

Z metodo `.head(n=5)` pogledamo prvih `n`, z metodo `.tail(n=5)` pa zadnjih `n` vrstic razpredelnice.

In [ ]:
filmi.head(10)

In [ ]:
filmi.tail()

Z indeksiranjem razpredelnice dostopamo do posameznih stolpcev. Če želimo več stolpcev, moramo za indeks podati seznam vseh oznak. Z rezinami pa dostopamo do izbranih vrstic.

In [ ]:
filmi['naslov']

In [ ]:
filmi[['naslov', 'ocena']]

In [ ]:
filmi[120:125]

Do vrednosti z indeksom `i` dostopamo z `.iloc[i]`, do tiste s ključem `k` pa z `.loc[k]`.

In [ ]:
filmi.iloc[120]

In [ ]:
filmi.loc[97576]

### Filtriranje in urejanje

Izbor določenih vrstic razpredelnice naredimo tako, da za indeks podamo stolpec logičnih vrednosti, ki ga dobimo z običajnimi operacijami. V vrnjeni razpredelnici bodo ostale vrstice, pri katerih je v stolpcu vrednost `True`.

In [ ]:
filmi.ocena >= 8

In [ ]:
filmi[filmi.ocena >= 8]

In [ ]:
filmi[(filmi.leto > 2010) & (filmi.ocena >= 8.5)]

Razpredelnico urejamo z metodo `.sort_values`, ki ji podamo ime ali seznam imen stolpcev, po katerih želimo urejati. Po želji lahko tudi povemo, kateri stolpci naj bodo urejeni naraščajoče in kateri padajoče.

In [ ]:
filmi.sort_values('leto')

In [ ]:
# najprej uredi padajoče po oceni, pri vsaki oceni pa še naraščajoče po letu
filmi.sort_values(['ocena', 'leto'], ascending=[False, True])

### Skupine

Z metodo `.groupby` ustvarimo razpredelnico posebne vrste, v katerem so vrstice združene glede na skupno lastnost.

In [ ]:
filmi_po_letih = filmi.groupby('leto')

In [ ]:
# povprečna ocena vsakega leta
filmi_po_letih['ocena'].mean()

In [ ]:
# če želimo, lahko združujemo tudi po izračunanih lastnostih
filmi['petletka'] = 5 * (filmi.leto // 5)
filmi

In [ ]:
filmi_po_petletkah = filmi.groupby('petletka')

Preštejemo, koliko filmov je bilo v vsakem petletki. Pri večini stolpcev dobimo iste številke, ker imamo v vsakem stolpcu enako vnosov. Če kje kakšen podatek manjkal, je številka manjša.

In [ ]:
filmi_po_petletkah.count()

Če želimo dobiti le število članov posamezne skupine, uporabimo metodo `.size()`. V tem primeru dobimo le stolpec, ne razpredelnice.

In [ ]:
filmi_po_petletkah.size()

Pogledamo povprečja vsake petletke. Dobimo povprečno leto, povprečno dolžino in oceno. Povprečnega naslova ne dobimo, ker se ga ne da izračunati, zato ustreznega stolpca ni.

In [ ]:
filmi_po_petletkah.mean()

## Risanje grafov

In [ ]:
# vključimo risanje grafov (če stvari začnejo delati počasneje, izklopimo možnost inline)
%matplotlib

Običajen graf dobimo z metodo `plot`. Uporabljamo ga, kadar želimo prikazati spreminjanje vrednosti v odvisnosti od zvezne spremenljivke

Naša hipoteza je, da so zlata leta filma mimo. Graf to potrjuje.

In [ ]:
filmi[filmi['ocena'] > 8.5].groupby('petletka').size().plot()

Razsevni diagram dobimo z metodo `plot.scatter`. Uporabljamo ga, če želimo ugotoviti povezavo med dvema spremenljivkama.

In [ ]:
filmi.plot.scatter('metascore', 'ocena')

In [ ]:
filmi[filmi.dolzina < 200].plot.scatter('dolzina', 'ocena')

Stolpčni diagram dobimo z metodo `plot.bar`. Uporabljamo ga, če želimo primerjati vrednosti pri diskretnih (običajno kategoričnih) spremenljivkah. Pogosto je koristno, da graf uredimo po vrednostih.

In [ ]:
filmi.sort_values('zasluzek', ascending=False).head(20).plot.bar(x='naslov', y='zasluzek')

## Stikanje podatkov

In [ ]:
osebe = pd.read_csv('obdelani-podatki/osebe.csv', index_col='id')
vloge = pd.read_csv('obdelani-podatki/vloge.csv')
zanri = pd.read_csv('obdelani-podatki/zanri.csv')

Razpredelnice stikamo s funkcijo `merge`, ki vrne razpredelnico vnosov iz obeh tabel, pri katerih se vsi istoimenski podatki ujemajo.

In [ ]:
vloge[vloge['film'] == 12349]

In [ ]:
zanri[zanri['film'] == 12349]

In [ ]:
pd.merge(vloge, zanri)

V osnovi vsebuje staknjena razpredelnica le tiste vnose, ki se pojavijo v obeh tabelah. Temu principu pravimo notranji stik (_inner join_). Lahko pa se odločimo, da izberemo tudi tiste vnose, ki imajo podatke le v levi tabeli (_left join_), le v desni tabeli (_right join_) ali v vsaj eni tabeli (_outer join_). Če v eni tabeli ni vnosov, bodo v staknjeni tabeli označene manjkajoče vrednosti. Ker smo v našem primeru podatke jemali iz IMDBja, kjer so za vsak film določeni tako žanri kot vloge, do razlik ne pride.

Včasih želimo stikati tudi po stolpcih z različnimi imeni. V tem primeru funkciji `merge` podamo argumenta `left_on` in `right_on`.

In [ ]:
pd.merge(pd.merge(vloge, zanri), osebe, left_on='oseba', right_on='id')

Poglejmo, katera osebe so nastopale v največ komedijah.

In [ ]:
zanri_oseb = pd.merge(pd.merge(vloge, zanri), osebe, left_on='oseba', right_on='id')
zanri_oseb[
    (zanri_oseb['zanr'] == 'Comedy') &
    (zanri_oseb['vloga'] == 'igralec')
].groupby(
    'ime'
).size(
).sort_values(
    ascending=False
).head(20)

## Analiza žanrov

### Povprečne ocene žanrov

Z združeno tabelo lahko izračunamo tudi povprečno oceno glede na žanr.

In [ ]:
pd.merge(
    zanri,
    filmi,
    left_on='film',
    right_on='id'
).groupby(
    'zanr'
).mean(
).sort_values(
    'ocena', ascending=False
)['ocena']

### Popularnost žanrov

Poglejmo, kako popularni so bili posamezni žanri skozi desetletja. Najprej vsakemu filmu dodajmo še stolpec z desetletjem.

In [ ]:
filmi['desetletje'] = 10 * (filmi['leto'] // 10)

Nato poglejmo, koliko je bilo filmov posameznega žanra v vsakem desetletju.

In [ ]:
zastopanost_zanrov = zanri.join(
    filmi, on='film'
).groupby(
    ['desetletje', 'zanr']
).size()
zastopanost_zanrov

Ker smo združevali po več lastnostih, smo dobili stolpec s hierarhičnim indeksom. Tega lahko pretvorimo v matriko z metodo `.unstack`.

In [ ]:
matrika_zastopanosti = zastopanost_zanrov.unstack()
matrika_zastopanosti

Ker nas zanima le popularnost žanra v posameznem desetletju, želimo za vsak stolpec izračunati razmerje števila filmov danega žanra glede na število vseh filmov. To storimo s pomočjo metode `.apply`, ki v dani razpredelnici dano funkcijo uporabi na vsaki vrstici ali stolpcu (če dodamo možnost `axis=1`). Če bi želeli funkcijo uporabiti na vsakem elementu stolpca, bi namesto `.apply` uporabili `.map`.

In [ ]:
matrika_popularnosti = matrika_zastopanosti.apply(lambda st: st / st.sum(), axis=1)
matrika_popularnosti

In [ ]:
matrika_popularnosti.plot(kind='area')

Zgornji graf je čisto nepregleden. Omejimo se le na 10 najpopularnejših žanrov.

In [ ]:
najpopularnejsi_zanri = zanri.groupby('zanr').size().sort_values(ascending=False)

In [ ]:
najpopularnejsi_zanri

Prve žanr ignorirajmo, ker je precej generičen.

In [ ]:
zanimivi_zanri = najpopularnejsi_zanri[1:11]

In [ ]:
zanri[zanri['zanr'].isin(list(zanimivi_zanri.index))]
zastopanost_zanrov = zanri[zanri['zanr'].isin(list(zanimivi_zanri.index))].join(
    filmi, on='film'
).groupby(
    ['desetletje', 'zanr']
).size()
matrika_zastopanosti = zastopanost_zanrov.unstack()
matrika_popularnosti = matrika_zastopanosti.apply(lambda st: st / st.sum(), axis=1)
matrika_popularnosti.plot(kind='area')

### Napovedovanje žanrov

In [ ]:
def koren_besede(beseda):
    beseda = ''.join(znak for znak in beseda if znak.isalpha())
    if not beseda:
        return '$'
    konec = len(beseda) - 1
    if beseda[konec] in 'ds':
        konec -= 1
    while konec >= 0 and beseda[konec] in 'aeiou':
        konec -= 1
    return beseda[:konec + 1]

def koreni_besed(niz):
    return pd.Series(list({
        koren_besede(beseda) for beseda in niz.replace('-', ' ').lower().split() if beseda
    }))

def koreni_filmov(nizi):
    return nizi[nizi.notnull()].apply(
        koreni_besed
    ).stack().reset_index(level=1, drop=True)

def verjetnosti_po_zanrih(dogodki_po_filmih):
    dogodki_po_filmih.name = 'dogodek'
    dogodki_po_zanrih = zanri.join(pd.DataFrame(dogodki_po_filmih), on='film')
    pogostost_dogodkov = pd.crosstab(dogodki_po_zanrih.dogodek, dogodki_po_zanrih.zanr)
    pogostost_dogodkov += 1
    return pogostost_dogodkov / pogostost_dogodkov.sum()

igralci = vloge[vloge.vloga == 'igralec'].set_index('film').oseba

In [ ]:
verjetnost_zanra = zanri.groupby('zanr').size() / len(filmi)
verjetnost_korena_opisa_pri_zanru = verjetnosti_po_zanrih(koreni_filmov(filmi.opis))

In [ ]:
verjetnost_korena_opisa_pri_zanru.Horror.sort_values(ascending=False).head(30)

In [ ]:
verjetnost_korena_opisa_pri_zanru.Crime.sort_values(ascending=False).head(30)

In [ ]:
verjetnost_korena_opisa_pri_zanru.Biography.sort_values(ascending=False).head(30)

In [ ]:
def doloci_zanre(opis):
    faktorji_zanrov = verjetnost_zanra
    faktorji_zanrov *= verjetnost_korena_opisa_pri_zanru[
        verjetnost_korena_opisa_pri_zanru.index.isin(
            koreni_besed(opis)
        )
    ].prod()
    faktorji_zanrov /= max(faktorji_zanrov)
    return faktorji_zanrov.sort_values(ascending=False).head(5)

In [ ]:
doloci_zanre('An alien space ship appears above Slovenia')

In [ ]:
doloci_zanre('A story about a young mathematician, who discovers her artistic side')